## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> For Seller to update:[Title_of_your_product](Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Resolve Clinical Health Information using the HPO taxonomy

- **Model**: `en.map_entity.hpo_resolver_pipe`
- **Model Description**: This advanced pipeline extracts human phenotype entities from clinical texts and utilizes the `sbiobert_base_cased_mli` Sentence Bert Embeddings to map these entities to their corresponding Human Phenotype Ontology (HPO) codes. It also returns associated codes from the following vocabularies for each HPO code: - MeSH (Medical Subject Headings)- SNOMED- UMLS (Unified Medical Language System ) - ORPHA (international reference resource for information on rare diseases and orphan drugs) - OMIM (Online Mendelian Inheritance in Man).


In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [2]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/base_serializers.py:28: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.22.4)
  import scipy.sparse


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

In [4]:
model_name = "en-map-entity-hpo-resolver-pipe"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.2xlarge"


## 2. Create a deployable model from the model package.

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker_session,
)


### Input Format

To use the model, you need to provide input in one of the following supported formats:

#### JSON Format

Provide input as JSON. We support two variations within this format:

1. **Array of Text Documents**: 
   Use an array containing multiple text documents. Each element represents a separate text document.

   ```json
   {
       "text": [
           "Text document 1",
           "Text document 2",
           ...
       ]
   }

    ```

2. **Single Text Document**:
   Provide a single text document as a string.


   ```json
    {
        "text": "Single text document"
    }
   ```

#### JSON Lines (JSONL) Format

Provide input in JSON Lines format, where each line is a JSON object representing a text document.

```
{"text": "Text document 1"}
{"text": "Text document 2"}
```

## 3. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A. Deploy the SageMaker model to an endpoint

In [6]:
# Deploy the model
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
)

---------!

Once endpoint has been created, you would be able to perform real-time inference.

In [7]:
import json
import pandas as pd
import os
import boto3

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


def process_data_and_invoke_realtime_endpoint(data, content_type, accept):

    content_type_to_format = {'application/json': 'json', 'application/jsonlines': 'jsonl'}
    input_format = content_type_to_format.get(content_type)
    if content_type not in content_type_to_format.keys() or accept not in content_type_to_format.keys():
        raise ValueError("Invalid content_type or accept. It should be either 'application/json' or 'application/jsonlines'.")

    i = 1
    input_dir = f'inputs/real-time/{input_format}'
    output_dir = f'outputs/real-time/{input_format}'
    s3_input_dir = f"{model_name}/validation-input/real-time/{input_format}"
    s3_output_dir = f"{model_name}/validation-output/real-time/{input_format}"

    input_file_name = f'{input_dir}/input{i}.{input_format}'
    output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    while os.path.exists(input_file_name) or os.path.exists(output_file_name):
        i += 1
        input_file_name = f'{input_dir}/input{i}.{input_format}'
        output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
    os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

    input_data = json.dumps(data) if content_type == 'application/json' else data

    # Write input data to file
    with open(input_file_name, 'w') as f:
        f.write(input_data)

    # Upload input data to S3
    s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_input_dir}/{os.path.basename(input_file_name)}", Body=bytes(input_data.encode('UTF-8')))

    # Invoke the SageMaker endpoint
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=input_data,
    )

    # Read response data
    response_data = json.loads(response["Body"].read().decode("utf-8")) if accept == 'application/json' else response['Body'].read().decode('utf-8')

    # Save response data to file
    with open(output_file_name, 'w') as f_out:
        if accept == 'application/json':
            json.dump(response_data, f_out, indent=4)
        else:
            for item in response_data.split('\n'):
                f_out.write(item + '\n')

    # Upload response data to S3
    output_s3_key = f"{s3_output_dir}/{os.path.basename(output_file_name)}"
    if accept == 'application/json':
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=json.dumps(response_data).encode('UTF-8'))
    else:
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=response_data)

    return response_data

### Initial Setup

In [8]:
docs = [
    """HPI: A 69-year-old white female with a history of depression, anxiety, admitted to the ABCD Hospital on February 6, 2007, for shortness of breath. The patient was consulted by Psychiatry for anxiety. I know this patient from a previous consult. During this recent admission, she was given Ativan 0.25 mg on a p.r.n. basis with relief after one to two hours. The patient was seen by Abc, MD, and Def, Ph.D. PAST MEDICAL HISTORY: The patient has a history of hypertension, hypothyroidism, cholelithiasis, Port-A-Cath placement, and hydronephrosis.""",
    """ A 5-month-old boy brought by his parents because of 2 days of cough. Mother took him when cough started 2 days go to Clinic. But cough got worse and he also started having fever yesterday at night. Mother did not measure it. REVIEW OF SYSTEMS: No vomiting. No diarrhea. No skin rash. No cyanosis. """]

sample_text = """She is followed by Dr. X in our office and has a history of severe tricuspid regurgitation. On 05/12/08, preserved left and right ventricular systolic function, aortic sclerosis with apparent mild aortic stenosis. She has previously had a Persantine Myoview nuclear rest-stress test scan completed at ABCD Medical Center in 07/06 that was negative. She has had significant mitral valve regurgitation in the past being moderate, but on the most recent echocardiogram on 05/12/08, that was not felt to be significant. She does have a history of significant hypertension in the past. She has had dizzy spells and denies clearly any true syncope. She has had bradycardia in the past from beta-blocker therapy."""

### JSON

In [9]:
input_json_data = {"text": sample_text}
data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )
pd.DataFrame(data["predictions"][0])

,ner_chunk,begin,end,ner_label,ner_confidence,code,resolution,all_k_codes,all_k_resolutions,all_k_distances,all_k_aux_labels
0,tricuspid regurgitation,67,89,HP,0.9826,HP:0005180,tricuspid regurgitation,HP:0005180:::HP:0010446:::HP:0001704:::HP:0001702:::HP:0030732:::HP:0031444:::HP:0011662:::HP:0031651:::HP:0031441:::HP:0031443:::HP:0011575:::HP:0001653:::HP:0010316:::HP:0001659:::HP:0031440:::HP:0001647:::HP:0011573,tricuspid regurgitation:::tricuspid stenosis:::tricuspid valve prolapse:::abnormality of the tricuspid valve:::tricuspid valve dysplasia:::dilatation of the tricuspid annulus:::tricuspid atresia:::abnormal tricuspid valve physiology:::abnormal tricuspid valve annulus morphology:::abnormal tricuspid valve leaflet morphology:::imperforate tricuspid valve:::mitral regurgitation:::ebstein anomaly of the tricuspid valve:::aortic valve regurgitation:::obsolete abnormal tricuspid valve morphology:::bicuspid aortic valve:::hypoplastic tricuspid valve,0.0000:::5.8977:::6.2173:::7.0236:::7.2061:::7.4600:::7.5063:::7.6319:::7.6426:::7.9385:::7.9596:::8.0861:::8.3368:::8.4977:::9.2676:::9.4231:::9.4823,"MSH:D014262||SNOMED:111287006||UMLS:C0040961||ORPHA:228410:::MSH:D014264||SNOMED:49915006||UMLS:C0040963||ORPHA:391641:::MSH:D014263||SNOMED:253383003||UMLS:C0040962||ORPHA:1101:::UMLS:C4025753||ORPHA:1759:::UMLS:C4255215||ORPHA:1724::::::MSH:D018785||SNOMED:253455004,63042009||UMLS:C0243002||ORPHA:391641::::::::::::UMLS:C4023292||ORPHA:1880:::MSH:D008944||SNOMED:48724000||UMLS:C0026266,C3551535||ORPHA:363700:::MSH:D004437||SNOMED:204357006||UMLS:C0013481||ORPHA:466791:::MSH:D001022||SNOMED:60234000||UMLS:C0003504||ORPHA:2181::::::MSH:C562388||SNOMED:72352009||UMLS:C0149630||ORPHA:1772:::UMLS:C4023294||ORPHA:2255"
1,aortic stenosis,197,211,HP,0.8053,HP:0001650,aortic stenosis,HP:0001650:::HP:0001682:::HP:0004381:::HP:0005174:::HP:0001691:::HP:0005145:::HP:0011611:::HP:0100545:::HP:0001659:::HP:0100546:::HP:0011645:::HP:0004942:::HP:0012397:::HP:0012727:::HP:0004380:::HP:0002616:::HP:0033538,aortic stenosis:::subvalvular aortic stenosis:::supravalvular aortic stenosis:::membranous subvalvular aortic stenosis:::muscular subvalvular aortic stenosis:::coronary artery stenosis:::aortic arch obstruction:::arterial stenosis:::aortic regurgitation:::carotid artery stenosis:::aortic sinus aneurysm:::aortic aneurysm:::aortic atherosclerosis:::thoracic aortic aneurysm:::aortic valve calcification:::aortic root aneurysm:::aortic annulus calcification,0.0000:::4.4104:::5.1986:::6.3503:::6.6584:::7.0477:::7.0632:::7.2026:::7.3230:::7.5884:::7.5889:::7.6998:::7.9957:::8.0672:::8.2905:::8.3184:::8.3248,MSH:D001024||SNOMED:60573004||UMLS:C0003507||ORPHA:536471:::MSH:D001020||SNOMED:204368006||UMLS:C0340375||ORPHA:1052:::MSH:D021921||SNOMED:268185002||UMLS:C0003499||ORPHA:391665:::UMLS:C1848978||ORPHA:3191:::UMLS:C3887554||OMIM:229310:::MSH:D023921||SNOMED:233970002||UMLS:C0242231||ORPHA:75565:::SNOMED:218728005||UMLS:C0152419||ORPHA:2255:::SNOMED:68109007||UMLS:C0038449||ORPHA:565:::MSH:D001022||SNOMED:60234000||UMLS:C0003504||ORPHA:2181:::MSH:D016893||SNOMED:64586002||UMLS:C0007282||ORPHA:536532:::SNOMED:54160000||UMLS:C2239253||ORPHA:1054:::MSH:D001014||SNOMED:67362008||UMLS:C0003486||ORPHA:1777:::SNOMED:81817003||UMLS:C0155733||ORPHA:412:::MSH:D017545||SNOMED:433068007||UMLS:C0162872||ORPHA:536467:::MSH:C562942||SNOMED:250978003||UMLS:C0428791||ORPHA:2072:::SNOMED:251036003||UMLS:C0238669||ORPHA:231160:::
2,mitral valve regurgitation,373,398,HP,0.91050005,HP:0001653,mitral valve regurgitation,HP:0001653:::HP:0001718:::HP:0001633:::HP:0001634:::HP:0031478:::HP:0031481:::HP:0001659:::HP:0031480:::HP:0005180:::HP:0004382:::HP:0033642:::HP:0005136:::HP:0011560:::HP:0031479:::HP:0011568:::HP:0011572,mitral valve regurgitation:::mitral valve stenosis:::abnormal mitral valve morphology:::mitral valve prolapse:::abnormal mitral valve annulus morphology:::abnormal mitral valve physiology:::aortic valve regurgitation:::abnormal mitral valve leaflet m

### JSON Lines

In [10]:
import json

def create_jsonl(records):

    if isinstance(records, str):
        records = [records]

    json_records = []

    for text in records:
        record = {
            "text": text
        }
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record) for record in json_records)

    return json_lines

In [11]:
input_jsonl_data = create_jsonl(sample_text)
data = process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"predictions": [{"ner_chunk": "tricuspid regurgitation", "begin": 67, "end": 89, "ner_label": "HP", "ner_confidence": "0.9826", "code": "HP:0005180", "resolution": "tricuspid regurgitation", "all_k_codes": "HP:0005180:::HP:0010446:::HP:0001704:::HP:0001702:::HP:0030732:::HP:0031444:::HP:0011662:::HP:0031651:::HP:0031441:::HP:0031443:::HP:0011575:::HP:0001653:::HP:0010316:::HP:0001659:::HP:0031440:::HP:0001647:::HP:0011573", "all_k_resolutions": "tricuspid regurgitation:::tricuspid stenosis:::tricuspid valve prolapse:::abnormality of the tricuspid valve:::tricuspid valve dysplasia:::dilatation of the tricuspid annulus:::tricuspid atresia:::abnormal tricuspid valve physiology:::abnormal tricuspid valve annulus morphology:::abnormal tricuspid valve leaflet morphology:::imperforate tricuspid valve:::mitral regurgitation:::ebstein anomaly of the tricuspid valve:::aortic valve regurgitation:::obsolete abnormal tricuspid valve morphology:::bicuspid aortic valve:::hypoplastic tricuspid valve"

### B. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [12]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Batch inference

In [13]:
import json
import os

input_dir = 'inputs/batch'
json_input_dir = f"{input_dir}/json"
jsonl_input_dir = f"{input_dir}/jsonl"

output_dir = 'outputs/batch'
json_output_dir = f"{output_dir}/json"
jsonl_output_dir = f"{output_dir}/jsonl"

os.makedirs(json_input_dir, exist_ok=True)
os.makedirs(jsonl_input_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)
os.makedirs(jsonl_output_dir, exist_ok=True)

validation_json_file_name = "input.json"

validation_jsonl_file_name = "input.jsonl"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/jsonl/"

def write_and_upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    if file_format == ".json":
        input_data = json.dumps(input_data)

    with open(file_name, "w") as f:
        f.write(input_data)

    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/batch/{file_format[1:]}/{os.path.basename(file_name)}",
        Body=(bytes(input_data.encode("UTF-8"))),
    )

In [14]:
input_jsonl_data = create_jsonl(docs)
input_json_data = {"text": docs}

write_and_upload_to_s3(input_json_data, f"{json_input_dir}/{validation_json_file_name}")

write_and_upload_to_s3(input_jsonl_data, f"{jsonl_input_dir}/{validation_jsonl_file_name}")

### JSON

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [16]:
from urllib.parse import urlparse

def process_s3_json_output_and_save(validation_file_name):

    output_file_path = f"{json_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    display(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        json.dump(data, f_out, indent=4)

In [17]:
process_s3_json_output_and_save(validation_json_file_name)

{'predictions': [[{'ner_chunk': 'shortness',
    'begin': 126,
    'end': 134,
    'ner_label': 'HP',
    'ner_confidence': '0.8931',
    'code': 'HP:0009826',
    'resolution': 'short limb',
    'all_k_codes': 'HP:0009826:::HP:0011927:::HP:0011219:::HP:0009824:::HP:0006385:::HP:0009381:::HP:0005773:::HP:0002984:::HP:0004279:::HP:0009821:::HP:0001773:::HP:0006402:::HP:0004322:::HP:0000331:::HP:0003502:::HP:0009237:::HP:0400005:::HP:0005792',
    'all_k_resolutions': 'short limb:::short digit:::short face:::short arms:::short legs:::short finger:::short forearm:::short radii:::short hands:::short forearms:::short feet:::short outer part of limbs:::small stature:::short chin:::mild short stature:::short little finger:::short ear:::short upper arms',
    'all_k_distances': '4.5894:::5.1137:::5.2076:::5.2497:::5.2998:::5.4811:::5.5209:::5.5477:::5.7461:::5.8376:::5.8744:::5.9193:::5.9920:::6.0456:::6.0716:::6.0995:::6.1374:::6.2386',
    'all_k_aux_labels': 'UMLS:C0239399||ORPHA:79243:::UM

### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [19]:
from urllib.parse import urlparse

def process_s3_jsonlines_output_and_save(validation_file_name):

    output_file_path = f"{jsonl_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        for item in data.split('\n'):
            f_out.write(item + '\n')

In [20]:
process_s3_jsonlines_output_and_save(validation_jsonl_file_name)

{"predictions": [{"ner_chunk": "shortness", "begin": 126, "end": 134, "ner_label": "HP", "ner_confidence": "0.8931", "code": "HP:0009826", "resolution": "short limb", "all_k_codes": "HP:0009826:::HP:0011927:::HP:0011219:::HP:0009824:::HP:0006385:::HP:0009381:::HP:0005773:::HP:0002984:::HP:0004279:::HP:0009821:::HP:0001773:::HP:0006402:::HP:0004322:::HP:0000331:::HP:0003502:::HP:0009237:::HP:0400005:::HP:0005792", "all_k_resolutions": "short limb:::short digit:::short face:::short arms:::short legs:::short finger:::short forearm:::short radii:::short hands:::short forearms:::short feet:::short outer part of limbs:::small stature:::short chin:::mild short stature:::short little finger:::short ear:::short upper arms", "all_k_distances": "4.5894:::5.1137:::5.2076:::5.2497:::5.2998:::5.4811:::5.5209:::5.5477:::5.7461:::5.8376:::5.8744:::5.9193:::5.9920:::6.0456:::6.0716:::6.0995:::6.1374:::6.2386", "all_k_aux_labels": "UMLS:C0239399||ORPHA:79243:::UMLS:C1860176,C4023124||ORPHA:397715:::UMLS

In [21]:
model.delete_model()

INFO:sagemaker:Deleting model with name: en-map-entity-hpo-resolver-pipe-2024-10-14-07-01-26-101


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

